In [1015]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from numpy import nan as NA


# P1 Carga y Limpieza de datos

### 1. Carga

In [1016]:
#Carga de los datos

df_all_13 = pd.read_csv("../Tarea-1-Datos/data/raw/w13/metrocuadrado_all_w13.csv", engine='python')
df_furnished_13 = pd.read_csv("../Tarea-1-Datos/data/raw/w13/metrocuadrado_all_w13.csv", engine='python')
df_all_14 = pd.read_csv("../Tarea-1-Datos/data/raw/w14/metrocuadrado_all_w14.csv", engine='python')
df_furnished_14 = pd.read_csv("../Tarea-1-Datos/data/raw/w14/metrocuadrado_furnished_w14.csv", engine='python')
df_all_15 = pd.read_csv("../Tarea-1-Datos/data/raw/w15/metrocuadrado_all_w15.csv", engine='python')
df_furnished_15 = pd.read_csv("../Tarea-1-Datos/data/raw/w15/metrocuadrado_furnished_w15.csv", engine='python')
df_all_16 = pd.read_csv("../Tarea-1-Datos/data/raw/w16/metrocuadrado_all_w16.csv", engine='python')
df_furnished_16 = pd.read_csv("../Tarea-1-Datos/data/raw/w16/metrocuadrado_furnished_w16.csv", engine='python')
df_all_17 = pd.read_csv("../Tarea-1-Datos/data/raw/w17/metrocuadrado_all_w17.csv", engine='python')
df_furnished_17 = pd.read_csv("../Tarea-1-Datos/data/raw/w17/metrocuadrado_furnished_w17.csv", engine='python')

#Agregamos variable categórica que diga si viene o no de un archivo "furnished"

df_all_13['furnished']=0
df_all_14['furnished']=0
df_all_15['furnished']=0
df_all_16['furnished']=0
df_all_17['furnished']=0
df_furnished_13['furnished']=1
df_furnished_14['furnished']=1
df_furnished_15['furnished']=1
df_furnished_16['furnished']=1
df_furnished_17['furnished']=1


#generamos un sólo DataFrame

frames = [df_all_13, df_furnished_13, df_all_14, df_furnished_14, df_all_15, df_furnished_15, df_all_16, df_furnished_16, df_all_17, df_furnished_17]

df = pd.concat(frames)
df1 = df
#Eliminamos filas duplicadas

df = df.drop_duplicates(['property_type|rent_type|location', 'price', 'n_rooms', 'n_bath',
       'surface', 'details', 'url', 'metrocuadrado_index'])

#Reindexamos y rdenamos para mejorar la eficiencia

df = df.reset_index(drop=True)
df.sort_index(inplace=True)

#¿cuantas filas furnished no estaban repetidas?

len(df[df["furnished"]>0])


1947

In [1017]:
#pasamos el precio de str a float
df["price"] = df["price"].replace('[\$,]', '', regex=True)
df["price"] = df["price"].replace('[\.,]', '', regex=True).astype(float)

#mismo ejercicio con la superficie y resto de columnas
df["surface"] = df["surface"].replace('[\ m2,]', '', regex=True).astype(float)

#si tiene más de 5 piezas o baños los agruparemos como que tienen 6
df["n_rooms"] = df["n_rooms"].replace('[\5+,]', '6', regex=True).astype(float)

df["n_bath"] = df["n_bath"].replace('[\5+,]', '6', regex=True).astype(float)


#df[df["n_rooms"].isnull()]["n_rooms"]

#Separamos la columna original en tres y la eliminamos luego

df["property_type"] = df['property_type|rent_type|location'].str.split(expand=True)[0]
df["property_type"] = df["property_type"].replace('[\,,]', '', regex=True)

df["rent_type"] = df['property_type|rent_type|location'].str.split(expand=True)[2]
df["rent_type"] = df["rent_type"].replace('[\,,]', '', regex=True)

def location(row):
    p = re.compile(", (.*) Bogot")
    result = p.search(row["property_type|rent_type|location"])
    if type(result)==type(None):
        return NA
    else:
        return result.group(1)
df['location'] = df.apply (lambda row: location(row), axis=1)
df = df[df['location'].notna()]

#dflocation = df['property_type|rent_type|location'].str.split().str[-5:-2]
#df["location"] = dflocation=dflocation.iloc[:,].apply(lambda x: ' '.join(x))

del df['property_type|rent_type|location']


In [1018]:
#creamos función para calcular el precio por metro cuadrado, excepto los casos en que la superficie sea 0

def price_per_m2 (row):
    if row['surface'] == 0 :
        return NA
    else:
        return row["price"]/row["surface"]
    
#creamos la nueva variable que guarde esta info

df['price per m2'] = df.apply (lambda row: price_per_m2(row), axis=1)

df["url"].loc[12000]

'https://www.metrocuadrado.com/inmueble/arriendo-casa-bogota-balcares-3-habitaciones-4-banos-4-garajes/9820-M2394337'

In [1019]:
#s="https://www.metrocuadrado.com/inmueble/arriendo-casa-bogota-balcares-3-habitaciones-4-banos-4-rajes/9820-M2394337"
#p = re.compile("(\d)-garaje")
#result = p.search(s)
#type(result)==type(None)

#creamos una funcion que busca el número de garajes en el str del url

def n_garajes(row):
    p = re.compile("(\d)-garaje")
    result = p.search(row["url"])
    if type(result)==type(None):
        return 0
    else:
        return result.group(1)
    
#creamos la nueva variable que guarde esta info

df['n_garajes'] = df.apply (lambda row: n_garajes(row), axis=1)


In [1020]:
df1=df.query('property_type=="Casa" and 80<=surface<120')
df2=df.query('property_type=="Casa" and 120<=surface<180')
df3=df.query('property_type=="Casa" and 180<=surface<240')
df4=df.query('property_type=="Casa" and 240<=surface<360')
df5=df.query('property_type=="Casa" and 360<=surface<=460')
df6=df.query('property_type=="Apartamento" and 40<=surface<60')
df7=df.query('property_type=="Apartamento" and 60<=surface<80')
df8=df.query('property_type=="Apartamento" and 80<=surface<=120')

df["product_type"]=0

for i in range(8):
    a=globals()["df" + str(i+1)]
    for j in a.index:
        df.at[j, 'product_type'] = i
        
        


In [1021]:
dfupz = pd.read_csv("../Tarea-1-Datos/data/asignacion_upz/barrio-upz.csv", engine='python')

In [1022]:
def pro_location(loc):
    if type(loc) == str:
        size = loc.strip()
        if len(size) != 0:
            return loc
    else:
        return 'NaN'

dfupz['pro_location'] = dfupz['pro_location'].apply(transforma_str)
dfupz['pro_location'] = dfupz['pro_location'].apply(str.upper)

In [1023]:
# Inicializamos la columna
df['upz'] = -1


def fusion_upz(df):
    
    locations = list(dfupz['pro_location'])
    upz_codes = list(dfupz['UPlCodigo'])
    
    for upz_code, location in zip(upz_codes, locations):
        ind = df[df['location'].str.contains(location)].index
        
        for i in ind:
            df.loc[i,'upz'] = upz_code
    
    return df


df = fusion_upz(df)


In [1024]:
print('Porcentaje con upz')
len(df[df['upz'] != -1])/len(df) * 100

Porcentaje con upz


93.52686019352686

In [1025]:
df_est_pob = pd.read_csv("../Tarea-1-Datos/data/estadisticas_upz/estadisticas_poblacion.csv", engine='python')
df_est_pob.drop(["Unnamed: 0", "nomupz"], axis=1, inplace=True)

df_ind_ins = pd.read_csv("../Tarea-1-Datos/data/estadisticas_upz/indice_inseguridad.csv", engine='python')
df_ind_ins.drop(["Unnamed: 0", "UPlNombre2"], axis=1, inplace=True)
df_ind_ins = df_ind_ins.rename(columns = {'UPlCodigo' : 'upz'})

df_por_ver = pd.read_csv("../Tarea-1-Datos/data/estadisticas_upz/porcentaje_areas_verdes.csv", engine='python')
df_por_ver.drop(["Unnamed: 0"], axis=1, inplace=True)
df_por_ver = df_por_ver.rename(columns = {'upz' : 'zone'})
df_por_ver = df_por_ver.rename(columns = {'cod_upz' : 'upz'})


def por_ver(row):
    return str("UPZ"+str(int(row["upz"])))

df_por_ver["upz"] = df_por_ver.apply (lambda row: por_ver(row), axis=1)


In [1026]:
df_info1 = df_est_pob.merge(df_por_ver, on = 'upz',how = 'left')
df_info = df_info1.merge(df_ind_ins, on = 'upz',how = 'left')
df = df.merge(df_info, on = 'upz',how = 'left')

In [1027]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17984 entries, 0 to 17983
Data columns (total 25 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   price                                    17984 non-null  float64
 1   n_rooms                                  17950 non-null  float64
 2   n_bath                                   17753 non-null  float64
 3   surface                                  17984 non-null  float64
 4   details                                  17920 non-null  object 
 5   url                                      17984 non-null  object 
 6   metrocuadrado_index                      17984 non-null  float64
 7   furnished                                17984 non-null  int64  
 8   property_type                            17984 non-null  object 
 9   rent_type                                17984 non-null  object 
 10  location                                 17984

In [1028]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17984 entries, 0 to 17983
Data columns (total 25 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   price                                    17984 non-null  float64
 1   n_rooms                                  17950 non-null  float64
 2   n_bath                                   17753 non-null  float64
 3   surface                                  17984 non-null  float64
 4   details                                  17920 non-null  object 
 5   url                                      17984 non-null  object 
 6   metrocuadrado_index                      17984 non-null  float64
 7   furnished                                17984 non-null  int64  
 8   property_type                            17984 non-null  object 
 9   rent_type                                17984 non-null  object 
 10  location                                 17984